# Neural Machine Translation

이번 주차 첫 번째 과제에 오신것을 환영합니다!

- 여러분은 이번 과제를 통해서 인간이 읽을 수 있는 날짜 표현("25th of June, 2009")을 기계가 인식할 수 있는 날짜 표현("2009-06-25")으로 변환하는 Neural Machine Translation(NMT) 모델을 구현합니다.
- 위 내용은 "attention model" 개념을 사용해 구현할 수 있으며, 이는 보다 정교한 sequence-to-sequence 모델의 한 종류입니다.

이번 과제에 필요한 패키지를 불러와 봅시다.

In [ ]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

## 1 - Translating human readable dates into machine dates

- 여기서 구축 할 모델은 영어에서 힌디어로 번역하는 것과 같이 한 언어에서 다른 언어로 번역하는 데 사용할 수 있습니다.
- 그러나 언어 번역에는 방대한 데이터 세트가 필요하며 일반적으로 GPU 교육에 며칠이 걸립니다.
- 대규모 데이터 세트를 사용하지 않고 이러한 모델을 실험하기 위해 더 간단한 "날짜 변환" 작업을 수행합니다.
- 인공 신경망에는 다양한 형식으로 작성된 날짜가 입력됩니다 (*예 : '1958 년 8 월 29 일', '1968 년 3 월 30 일', '1987 년 6 월 24 일'*)
- 신경망은 이 날짜를 기계가 읽을 수 있는 표준화 된 날짜 형식으로 변환합니다 (*예 : "1958-08-29", "1968-03-30", "1987-06-24"*).
- 우리는 신경망이 YYYY-MM-DD의 일반적인 기계 판독 형식으로 날짜를 출력하는 방법을 배우게 할 것입니다.

### 1.1 - Dataset

사람이 읽을 수있는 10,000개의 날짜와 이에 상응하는 표준화 된 기계가 읽을 수 있는 날짜로 구성된 데이터 세트를 통해 모델을 학습합니다. 다음 셀을 실행하여 데이터 세트를 로드하고 몇 가지 예시를 출력 해 보겠습니다.

In [ ]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

In [ ]:
dataset[:10]

불러온 데이터는 다음과 같습니다.
- `dataset` : (human readable date, machine readable date) 형태의 튜플 데이터들을 담고 있는 리스트입니다.
- `human_vocab` : human-readable date와 정수 값의 인덱스를 맵핑하고 있는 파이썬 딕셔너리입니다.
- `machine_vocab` : machine-readable date와 정수 값인 인덱스를 맵핑하고 있는 파이썬 딕셔너리 입니다.
  - **주의** : 이 인덱스는 반드시 `human_vocab`의 인덱스와 일치하지 않습니다.
- `inv_matchine_vocab` : 문자열과 인덱스들을 거꾸로 맵핑하는, `machine_vocab`의 뒤집어진 딕셔너리입니다.

데이터를 전처리하고 원시 텍스트 데이터를 인덱스 값에 매핑 해 보겠습니다.
- Tx = 30으로 설정하겠습니다.
  - Tx는 사람이 읽을 수있는 날짜의 최대 길이라고 가정합니다.
  - 더 긴 입력이 들어오면 잘라야합니다.
- Ty = 10으로 설정하겠습니다.
  - "YYYY-MM-DD"는 10 자입니다.

In [ ]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

이제 우리는 다음의 데이터를 가지고 있습니다.
- `X` : 훈련 데이터 세트에서, 인간이 읽을 수 있는 형식의 날짜를 전처리한 데이터입니다.
  - X의 각 문자열은 정수형 index로 대체되었습니다. `human_vocab`를 사용하여 이 index에 맵핑된 문자열을 얻을 수 있습니다.
  - 각각의 날짜는 고정된 길이 $T_x$에 맞추기 위해, (<pad>) 라고 하는 특별한 문자로 패딩되었습니다.
  - `X.shape = (m, Tx)`에서, m은 한 배치에 들어 있는 훈련 데이터의 갯수입니다.
- `Y` : 기계가 읽을 수 있는 형식의 날짜를 전처리한 데이터 입니다.
  - Y의 각 문자열은 `machine_vocab`에 맵핑된 index 값으로 대체되었습니다.
  - `Y.shape = (m, Ty)`
- `Xoh` : `X`의 one-hot 인코딩 벡터입니다.
  - X의 각 index는 one-hot 인코딩 버전으로 변환되었습니다.(예를 들어 index가 2라면, one-hot 버전은 2번째 포지션이 1이고 나머지 포지션이 0인 형식을 취합니다.)
  - `Xoh.shape = (m, Tx, len(human_vocab))`
- `Yoh` : `Y`의 one-hot 인코딩 벡터입니다.
  - Y의 각 index를 one-hot 인코딩 버전으로 변환시킵니다.
  - `Yoh.shape = (m, Tx, len(machine_vocab))`
  - `len(machine_vocab) = 11`에서, 기계는 0-9까지의 10가지 숫자와 '-'를 포함한 총 11개의 텍스트 심볼을 읽을 수 있습니다.


- 전처리된 데이터 셋의 몇 가지 다른 예시도 살펴 보겠습니다.
- 데이터 세트를 탐색하고 소스/타겟 날짜가 전처리되는 방식을 확인하려면 아래 셀의 'index'를 자유롭게 변경하세요.

In [ ]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

## 2 - Neural machine translation with attention

* 책의 문단을 프랑스어에서 영어로 번역해야한다면 전체 문단을 읽지 않고 책을 닫고 번역합니다.
* 번역 과정 중에도, 여러분은 읽고/다시 읽고 적어 놓은 영어 부분에 해당하는 프랑스어 단락 부분에 집중합니다.
* attention 매커니즘은 Neural Machine Translation 모델에 어느 단계에서든 주의를 기울여야하는 위치를 알려줍니다.

### 2.1 - Attention mechanism

이 파트에서, 여러분은 동영상 강의에서 소개된 attention 매커니즘을 구현합니다.

- 아래에 모델이 어떻게 동작하는지에 대한 그림이 있습니다.
  - 왼쪽 그림은 attention 모델을 보여줍니다.
  - 오른쪽 그림은, 하나의 "attention" 단계가 변수 $\alpha^{\langle t, t' \rangle}$를 계산하는 과정을 보여줍니다.
  - attention 변수 $\alpha^{\langle t, t' \rangle}$는 매 time step ($t=1, \ldots, T_y$)마다, 맥락 변수 $context^{\langle t \rangle}$를 구하는데 사용됩니다.

<table>
<td> 
<img src="arts/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="arts/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<center>그림 1 - attention 모델을 사용한 Neural machine translation</center>

이 모델에서 여러분이 기억할 몇 가지 특성들이 있습니다.

#### Pre-attention and Post-attention LSTMs on both sides of attention mechanism

- 왼쪽 그림에서 볼 수 있듯이, 이 모델에는 두 가지 독립적인 LSTM 모델이 있습니다 : Pre-attention과 Post-attention LSTM 모델입니다.
- **Pre-attention Bi-LSTM** : 이 모델은 사진 하단에 있는 양방향의 LSTM 모델이고, attention 매커니즘 이전에 계산됩니다.
  - attention mechanism은 왼쪽 다이어그램의 중간에 보이는 그림입니다.
  - Bi-LSTM인 pre-attention은 $T_x$ time step을 거쳐서 수행됩니다.
- **Post-attention LSTM** : 이 모델은 hidden state $s^{\langle t \rangle}$와 cell state $c^{\langle t \rangle}$를 한 time step에서 다른 time step으로 전달합니다.

#### An LSTM has both a hidden state and cell state


* 강의 비디오에서는 Post-attention 시퀀스 모델에 기본적인 RNN 만 사용했습니다.
  * 이것은 RNN에 의해 ​​캡처 된 상태가 hidden state $ s ^ {\langle t \rangle} $ 만 출력했음을 의미합니다.
* 이 과제에서는 기본 RNN 대신 LSTM을 사용하고 있습니다.
  * 따라서 LSTM은 hidden state $ s ^ {\langle t \rangle} $와 cell state $ c ^ {\langle t \rangle} $를 모두 갖습니다.

#### Each time step does not use predictions from the previous time step


* 코스 초반의 텍스트 생성 예제와 달리, 이 모델에서는 $ t $ time step의 post-attention LSTM이 이전 time step의 예측 $ y ^ {\langle t-1 \rangle} $을 입력으로 사용하지 않습니다.
* Time step $t$에서의 post-attention LSTM 는 hidden state $ s ^ {\langle t \rangle} $ 및 cell state $ c ^ {\langle t \rangle} $ 만 입력으로 사용합니다.
* 언어 생성 (인접한 문자가 높은 상관 관계가 있는 경우)과 달리 YYYY-MM-DD 날짜의 경우 이전 문자와 다음 문자 사이에 강한 종속성이 없기 때문에 이러한 방식으로 모델을 설계했습니다.

#### Concatenation of hidden states from the forward and backward pre-attention LSTMs


- $ \overrightarrow {a} ^ {\langle t \rangle} $ : 정방향 pre-attention LSTM의 hidden state.
- $ \overleftarrow {a} ^ {\langle t \rangle} $ : 역방향, pre-attention LSTM의 hidden state.
- $ a ^ {\langle t \rangle} = [\overrightarrow {a} ^ {\langle t \rangle}, \overleftarrow {a} ^ {\langle t \rangle}] $ : Pre-attention Bi-LSTM의 정방향 $ \ \overrightarrow {a} ^ {\langle t \rangle} $ 및 역방향 $ \overleftarrow {a} ^ {\langle t \rangle} $를 연결한 최종 활성화 값.

#### Computing "energies" $e^{\langle t, t' \rangle}$ as a function of $s^{\langle t-1 \rangle}$ and $a^{\langle t' \rangle}$

- "Attention Model" 동영상 강의의 6:45부터 8:16 사이의 내용을 기억해봅시다. "e"라는 변수는 $a^{\langle t \rangle}$ 와 $s^{\langle t-1 \rangle}$의 함수로 정의합니다.
  - "e"는 "energy" 변수로 부릅니다.
  - $s^{\langle t-1 \rangle}$ post-attention LSTM의 hidden state입니다.
  - $a^{\langle t' \rangle}$ 는 pre-attention LSTM의 hidden state입니다.
  - $s^{\langle t-1 \rangle}$과 $a^{\langle t \rangle}$는 간단한 인공 신경망에 투입되어, 출력값인 $e^{\langle t, t' \rangle}$를 구하는 함수를 학습합니다.
  - $e^{\langle t, t' \rangle}$ 값은 $y^{\langle t \rangle}$가 특정 $a^{\langle t' \rangle}$를 고려해야 하는 크기를 나타내는 attention 변수인 $a^{\langle t, t' \rangle}$를 계산하는데 사용됩니다.





- 그림 1의 오른쪽에 나타난 다이어그램에선, $s^{\langle t-1 \rangle}$를 $T_x$ 번 만큼 복사하기 위해 `RepeatVector` 노드를 사용합니다.
- 이후 `Concatnation`을 사용하여 $s^{\langle t-1 \rangle}$과 $a^{\langle t \rangle}$를 이어붙입니다.
- $s^{\langle t-1 \rangle}$과 $a^{\langle t \rangle}$가 이어붙여진 값은 "Dense" 레이어에 투입되고, 출력으로 $e^{\langle t, t' \rangle}$를 계산합니다.
- $e^{\langle t, t' \rangle}$는 softmax 함수를 통과하여 $\alpha^{\langle t, t' \rangle}$를 계산하는데 사용됩니다.
- 위의 다이어그램은 명시적으로 $e^{\langle t, t' \rangle}$를 표시하지 않지만, $e^{\langle t, t' \rangle}$는 그림 1의 오른쪽에 있는 Dense 레이어와 softmax 레이어 아래에 있습니다.
- 케라스에서 `RepeatVector`와 `Concatenation`를 사용하는 방법을 아래에서 설명하겠습니다.

### Implementation Details

Neural Translator를 구해봅시다. `one_step_attention()`과 `model()` 두 함수를 구현하는 것으로 시작해 보겠습니다.

#### one_set_attention()
- 특정 time step $t$에서의 `one_step_attention()` 함수의 입력은 다음과 같습니다.
  - $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$ : pre-attention Bi-LSTM의 모든 hidden state
  - $s^{<t-1>}$ : post-attention LSTM의 이전 hidden state
- `one_step_attention()` 함수는 다음을 계산합니다.
  - $[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$ : attention 가중치
  - $context^{ \langle t \rangle }$ : context 벡터
  $$context^{<t>} = \sum_{t' = 1}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 


#### Clarifing 'context' and 'c'
- 강의에서, 컨텍스트는 $c^{\langle t \rangle}$로 표기됩니다.
- 이번 과제에서는, 컨텍스트 변수를 we are calling the context $context^{\langle t \rangle}$로 표기하겠습니다.
  - 이는 post-attention LSTM의 내부 메모리 cell 변수 $c^{\langle t \rangle}$와 구분하기 위해서입니다. 


#### Implement `one_step_attention`

**연습 문제** : `one_step_attention()` 함수를 구현하세요.

- `model()` 함수는 반복문을 사용해서 `one_step_attention()` $T_y$의 레이어를 호출합니다.
- 모든 $T_y$의 복사본이 동일한 가중치를 가지는 것이 중요합니다.
  - 매번 가중치를 다시 초기화해선 안됩니다.
  - 다시 말해, 모든 $T_y$ 단계에서 가중치가 공유되어야 합니다.
- 케라스에서 공유 가능한 가중치로 레이어를 구현하는 방법은 다음과 같습니다.
  1. `one_step_attention` 함수 외부에 있는 범위에서 레이어를 정의합니다. 예를 들어 객체를 전역 변수로 정의하세요.
    - `model` 함수 내부에서 레이어 객체를 정의해도 기술적으로는 잘 동작합니다. `model`함수 내부에서 `one_step_attention`함수를 호출하기 때문입니다. 그러나 이번 과제에선 채점의 편의를 위해 레이어 객체를 전역변수로 선언하겠습니다. 자동 채점 앨고리즘은 이 변수를 전역 변수로 예상하고 채점합니다.
  2. 입력을 넘길 때 이 전역 변수 객체를 호출합니다.
- 필요한 레이어를 전역 변수로 정의했습니다.
  - 전역 변수 객체를 생성하려면 아래 코드를 실행하세요.
  - 채점 알고리즘은 주어진 변수 이름을 그대로 사용합니다. 따라서 전역 변수의 이름을 임의로 바꾸지 마세요.
- 이 공유 가능 레이어에 대해 자세히 알아보려면 케라스의 문서를 확인하세요. 레이어는 함수입니다. 아래는 일련의 함수를 어떻게 호출하는지에 대한 내용입니다.
    - [RepeatVector()](https://keras.io/layers/core/#repeatvector)
```Python
var_repeated = repeat_layer(var1)
```
    - [Concatenate()](https://keras.io/layers/merge/#concatenate)   
```Python
concatenated_vars = concatenate_layer([var1,var2,var3])
```
    - [Dense()](https://keras.io/layers/core/#dense)  
```Python
var_out = dense_layer(var_in)
```
    - [Activation()](https://keras.io/layers/core/#activation)  
```Python
activation = activation_layer(var_in)  
```
    - [Dot()](https://keras.io/layers/merge/#dot)  
```Python
dot_product = dot_layer([var1,var2])
```

In [ ]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [ ]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = None
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = None
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = None
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = None
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = None
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = None
    ### END CODE HERE ###
    
    return context

`model()`함수를 구현한 이후 `one_step_attention()`의 예상 출력을 확인할 수 있습니다.

#### model

- `model`은 먼저 Bi-LSTM을 통해 입력을 실행하여 $[a ^ {<1>}, a ^ {<2>}, ..., a ^ {<T_x>}] $를 얻습니다.
- 이후 `model` 함수는 반복문을 사용하여 `one_step_attention`을 $T_y$번 호출합니다. 이 루프가 반복될 때마다.
  - 계산된 컨텍스트 벡터 $context^{<t>}$를 post-attention LSTM에 투입합니다.
  - softmax 활성화 함수와 dense layer를 통해서 post-attention LSTM의 출력을 실행합니다.
  - softmax 함수는 예측 결과 $\hat{y}^{<t>}$를 계산합니다.

**연습 문제** : 그림 1과 위의 텍스트에 설명 된대로 `model()` 함수를 구현합니다. 다시 말하지만,`model()` 에서 사용할 가중치를 공유 할 전역 레이어를 정의했습니다.

In [ ]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.  
# For the purposes of passing the automatic grader
# please do not modify this global variable.  This will be corrected once the automatic grader is also updated.
post_activation_LSTM_cell = LSTM(n_s, return_state = True) # post-attention LSTM 
output_layer = Dense(len(machine_vocab), activation=softmax)

이제 반복문에서 이러한 레이어를 $ T_y $ 번 사용하여 출력을 생성 할 수 있으며 해당 파라미터는 매 반복마다 다시 초기화되지 않습니다. 다음 단계를 수행해야합니다.

1. 입력`X`를 양방향 LSTM으로 전파합니다.
  * [양방향](https://keras.io/layers/wrappers/#bidirectional)
  * [LSTM](https://keras.io/layers/recurrent/#lstm)
  * LSTM이 마지막 hidden state 대신 전체 시퀀스를 반환하기를 원한다는 것을 기억하십시오.
  * 샘플 코드 :
  ```python
  sequence_of_hidden_states = Bidirectional(LSTM(units=..., return_sequences=...))(the_input_X)
  ```
2. $ t = 0, \cdots, T_y-1 $에 대해 반복합니다.
  1. `one_step_attention ()`을 호출하여 hidden state를 전달합니다. $context^{<t>}$를 계산하기 위해 양방향 LSTM인 pre-attention LSTM의 hideen state $[a^{\langle 1 \rangle},a^{\langle 2 \rangle}, ..., a^{ \langle T_x \rangle}]$와 post-attention LSTM의 이전 hidden state $s^{<t-1>}$를 입력받습니다.
  2. $context^{<t>}$를 post-attention LSTM cell에 입력합니다.
    - 이 LSTM의 이전 hidden state $ s ^ {\langle t-1 \rangle} $ 및 cell state $ c ^ {\langle t-1 \rangle} $를 전달해야합니다.
    - 아래 샘플 코드와 같이 하면 새로운 hidden state $s^{\langle t \rangle}$와 cell state $c^{\langle t \rangle}$가 출력됩니다.
    - 샘플 코드 :
    ```python
    next_hidden_state, _ , next_cell_state = 
            post_activation_LSTM_cell(inputs=..., initial_state=[prev_hidden_state, prev_cell_state])
    ```
  여기서 레이어는 실제 post attention LSTM cell입니다. 자동 채점을 위해 전역 변수의 이름을 임의로 수정하지 마세요. 
  3. $ s ^ {<t>} $에 dense 레이어와 softmax 레이어를 적용하고 결과를 얻습니다.
    - 샘플 코드 : 
    ```python
        output = output_layer(inputs=...)
    ```
  4. 이 결과를 `outputs` 리스트에 추가합니다.
3. 케라스 모델 인스턴스를 생성합니다.
  - 세 가지 인자를 입력합니다.
    - `X` : $(T_x, humanVocabSize)$의 shape를 가진 one-hot 인코딩 벡터입니다.
    - $ s ^ {\langle 0 \rangle} $ : post-attention LSTM의 초기 hidden state
    - $ c ^ {\langle 0 \rangle} $) : post-attention LSTM의 초기 cell state
  - 이 모델의 출력 결과는 `outputs` 리스트입니다.
  - 샘플 코드 :
  ```python
    model = Model(inputs=[...,...,...], outputs=...)
  ```


In [ ]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    a = None
    
    # Step 2: Iterate for Ty steps
    for t in range(None):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = None
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = None
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = None
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        None
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = None
    
    ### END CODE HERE ###
    
    return model

다음 셀을 실행하여 모델을 만듭니다.


In [ ]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

#### Troubleshooting Note

* 처음에 "모델"을 잘못 구현 한 후 반복되는 오류가 발생하지만 오류를 수정했다고 생각되는 경우에도 모델을 빌드 할 때 오류 메시지가 계속 표시 될 수 있습니다.
* 해결책은 모든 데이터를 저장하고 커널을 다시 시작 (또는 노트북을 종료 한 다음 다시 시작)하고 셀을 다시 실행하는 것입니다.

예상 출력과 일치하는지 확인하기 위해 완성된 모델에 대한 요약을 가져옵니다.

In [ ]:
model.summary()

**모범 답안**:

아래는 여러분이 확인하실 수 있는 모델의 요약 자료입니다.
<table>
    <tr>
        <td>
            **Total params:**
        </td>
        <td>
         52,960
        </td>
    </tr>
        <tr>
        <td>
            **Trainable params:**
        </td>
        <td>
         52,960
        </td>
    </tr>
            <tr>
        <td>
            **Non-trainable params:**
        </td>
        <td>
         0
        </td>
    </tr>
                    <tr>
        <td>
            **bidirectional_1's output shape **
        </td>
        <td>
         (None, 30, 64)  
        </td>
    </tr>
    <tr>
        <td>
            **repeat_vector_1's output shape **
        </td>
        <td>
         (None, 30, 64) 
        </td>
    </tr>
                <tr>
        <td>
            **concatenate_1's output shape **
        </td>
        <td>
         (None, 30, 128) 
        </td>
    </tr>
            <tr>
        <td>
            **attention_weights's output shape **
        </td>
        <td>
         (None, 30, 1)  
        </td>
    </tr>
        <tr>
        <td>
            **dot_1's output shape **
        </td>
        <td>
         (None, 1, 64)
        </td>
    </tr>
           <tr>
        <td>
            **dense_3's output shape **
        </td>
        <td>
         (None, 11) 
        </td>
    </tr>
</table>

#### Compile the model

* Keras에서 모델을 생성 한 후 이를 컴파일하고 사용할 손실 함수, 최적화 함수 및 메트릭을 정의해야합니다.
  * 손실 함수 : `categoical_crossentropy`
  * 최적화 함수 : [Adam](https://keras.io/optimizers/#adam) [optimizer](https://keras.io/optimizers/#usage-of-optimizers)
    - learning rate = 0.005 
    - $\beta_1 = 0.9$
    - $\beta_2 = 0.999$
    - decay = 0.01  
  * metric: 'accuracy'

<br>

샘플 코드
```Python
optimizer = Adam(lr=..., beta_1=..., beta_2=..., decay=...)
model.compile(optimizer=..., loss=..., metrics=[...])
```

In [ ]:
### START CODE HERE ### (≈2 lines)
opt = None
None
### END CODE HERE ###

#### Define inputs and outputs, and fit the model

마지막 단계는 모든 입력과 출력을 모델에 맞게 정의하는 것입니다.
- 훈련 데이터 세트가 포함 된 $ (m = 10000, T_x = 30) $ 모양의 입력 X가 있습니다.
- 'post_attention_LSTM_cell'을 0으로 초기화하려면 's0'과 'c0'을 만들어야합니다.
- 구현한 `model ()`이 주어지면 10 개의 데이터를 가지는 shape (m, T_y)리스트를 출력해야 합니다.
     - `outputs [i] [0], ..., outputs [i] [Ty]`목록은 $ i ^ {th} $ 훈련 데이터 (`X [i]`에 해당하는 실제 레이블 (문자))을 나타냅니다. ).
     - `outputs [i] [j]`는 $ i ^ {th} $ 훈련 데이터에서 $ j ^ {th} $ 문자의 실제 레이블입니다.

In [ ]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

이제 모델을 맞추고 1 epoch 동안 실행 해 보겠습니다.

훈련하는 동안 출력의 10 개 위치 각각에 대한 손실과 정확도를 볼 수 있습니다. 아래 표는 배치에 2 개의 훈련 데이터가 있는 경우 정확도가 어떻게 될 수 있는지에 대한 예를 제공합니다.

<img src="arts/table.png" style="width:700;height:200px;">
<center>따라서 `dense_2_acc_8 : 0.89`는 출력의 7 번째 문자를 현재 데이터 배치에서 89%의 시간 동안 정확하게 예측하고 있음을 의미합니다.</center>

이 모델을 더 오래 실행하고 가중치를 저장했습니다. 다음 셀을 실행하여 가중치를 로드합니다. (몇 분 동안 모델을 훈련하면 비슷한 정확도의 모델을 얻을 수 있지만 미리 훈련된 모델을 불러오면 시간이 절약됩니다.)

In [ ]:
model.load_weights('models/model.h5')

이제 새로운 데이터에서 결과를 볼 수 있습니다.

In [ ]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output),"\n")

이러한 훈련 데이터를 변경하여 자신만의 예제로 테스트 할 수도 있습니다. 다음 부분에서는 attention mechanism이 수행하는 작업, 즉 특정 출력 문자를 생성 할 때 네트워크가 주의를 기울이는 입력 부분에 대해 더 잘 이해할 수 있습니다.

## 3 - Visualizing Attention (Optional / Ungraded)

문제의 출력 길이가 10으로 고정되어 있으므로 10 개의 서로 다른 softmax unit을 사용하여 이 작업을 수행하여 10 개의 출력 문자를 생성 할 수도 있습니다. 그러나주의 모델의 한 가지 장점은 출력의 각 부분 (예 : 월)이 입력의 작은 부분(월을 나타내는 입력의 문자)에만 의존해야 한다는 것을 알고 있다는 것입니다. 출력의 각 부분이 입력의 어느 부분을 보고 있는지 시각화 할 수 있습니다.

"2018 년 5 월 9 일 토요일"을 "2018-05-09"로 번역하는 작업이 있다고 생각해 보세요. 계산 된 $ \alpha ^ {\langle t, t '\rangle} $를 시각화하면 다음과 같이됩니다.

<img src="arts/date_attention.png" style="width:600;height:300px;"> 
<caption><center>그림 8 : Full Attention Map</center></caption>

<br>

출력이 입력의 "토요일"부분을 어떻게 무시하는지 확인하십시오. 출력 time step 중 어느 것도 입력의 해당 부분에 많은 주의를 기울이지 않습니다. 또한 9가 09로 번역되었고 5월이 05로 올바르게 번역되었음을 알 수 있습니다. 출력은 번역에 필요한 입력 부분에 주의를 기울입니다. "년도" 값은 대부분 "2018"을 생성하기 위해 입력의 "18"에 주의를 기울여야합니다.

### 3.1 - Getting the attention weights from the network

이제 신경망의 attention value를 시각화 해 보겠습니다. 신경망을 통해 데이터를 정방향 계산 한 다음 $ \alpha ^ {\langle t, t '\rangle} $의 값을 시각화합니다.

attention value가 어디에 있는지 알아 내기 위해 먼저 모델 요약을 출력 해 보겠습니다.

In [ ]:
model.summary()

위의 `model.summary()`출력을 살펴보세요. 'dot_2'가 $ t = 0, \ldots, T_y-1 $ 단계마다 컨텍스트 벡터를 계산하기 전에 'attention_weights'라는 레이어가 shape (m, 30, 1)의 'alpha'를 출력하는 것을 볼 수 있습니다. 이 레이어에서 attention weight를 가져 오겠습니다.

`attention_map()`함수는 모델에서 주의 값을 가져 와서 그래프를 그리는 역할을 합니다.

In [ ]:
attention_map = plot_attention_map(model, human_vocab, inv_machine_vocab, "Tuesday 09 Oct 1993", num = 7, n_s = 64);

생성 된 그래프에서 예측 된 출력의 각 문자에 대한 attention weight 값을 관찰 할 수 있습니다. 이 플롯을 조사하고 신경망이 주의를 기울이고있는 위치가 실제로 의미가 있는지 확인하십시오.

날짜 번역 응용 프로그램에서 대부분의 시간 관련 attention은 연도를 예측하는 데 도움이 되며 날짜 나 월을 예측하는 데 큰 영향을 주지 않습니다.

### Congratulations!

이 과제를 끝마쳤습니다!

## Here's what you should remember

- Machine translation model을 사용하여 한 시퀸스를 사용해 다른 시퀸스로 맵핑할 수 있습니다. 이는 언어 번역(예 : 프랑스어 -> 영어) 번역 뿐만 아니라 날짜 형식과 같은 작업에도 유용합니다.
- Attention mechanism은 출력의 특정 부분을 생성할 때 입력의 가장 관련성 있는 부분에 집중할 수 있습니다.
- Attention mechanism을 사용하는 인공 신경망은 $T_x$ 길이의 입력에서, $T_y$ 길이의 출력으로 변환할 수 있습니다. 여기서 $T_x$와 $T_y$는 다를 수 있습니다.
- Attention weight $\alpha^{\langle t,t' \rangle}$를 시각화하여, 각 출력을 생성하는 동안 신경망이 주의를 기울이고 있는 부분을 확인할 수 있습니다.

이 과제를 마치신 것을 축하드립니다! 이제 attention model을 구현하고 이를 사용하여 한 시퀀스에서 다른 시퀀스로의 복잡한 매핑을 학습 할 수 있습니다.